# Machine Learning Workshop

Here we will walk through an example of a machine learning workflow following five steps:

1. Formulating a question
2. Data collection/cleaning
3. Feature engineering
4. Model selection and hyperparameter tuning
5. Model application

For more information on the Shiu Lab's ML pipeline, check out the [README](https://github.com/ShiuLab/ML-Pipeline/blob/master/README.md).


### Set up

Check out this [guide](https://github.com/ShiuLab/ML-Pipeline/blob/master/Tutorial/README.md) to learn how to set up Jupyter notebook and the software needed to run the Shiu Lab's ML pipeline.


In [2]:
import pandas as pd

## 1. Formulating a question

Topic: Specialized metabolism in tomato.

Possible Questions:
- How well can we predict if a gene is involved in specialized or general metabolism?
- What features are most predictive of specialized metabolism?
- Which unknown genes are likely involved in specialized metabolism?
- Can information from a model system like Arabidopsis help us learn about specialized metabolism in tomato?


What do we need:

- Training data





## 2. Data collection/cleaning

Steps:
   - A. Load in and look at the data
   - B. Remove or impute n/a values and convert categorical features into a format suitable for ML modeling
   - C. Define the testing set (these data will not be used at any stage during parameter selection or training)


In [36]:
## A. Load in and look at the data

d = pd.read_csv('data.txt', sep='\t', index_col = 0)

print('Shape of data (rows, cols):')
print(d.shape)

print('\nSnapshot of data:')
print(d.head())

print('\n List of class labels')
print(d['Class'].unique())

Shape of data (rows, cols):
(2874, 565)

Snapshot of data:
                Class  Crubella_183_v1.0.csv  FamilySize FamilySize_cat  \
YP_008563134      gen                    0.0    0.010582         medium   
XP_010327628      gen                    0.0    0.000000          small   
XP_010327620  special                    0.0    0.052910         medium   
XP_010327578      gen                    0.0         NaN            NaN   
XP_010327494      gen                    1.0    0.021164         medium   

              Transferase  SQS_PSY  Exo_endo_phos  IPT  Paxillaris_medKaKs  \
YP_008563134          0.0      NaN            0.0  0.0            0.179007   
XP_010327628          NaN      NaN            0.0  0.0            0.179007   
XP_010327620          0.0      NaN            0.0  0.0            0.237404   
XP_010327578          0.0      NaN            0.0  0.0            0.237404   
XP_010327494          0.0      0.0            0.0  0.0            0.092387   

              GalKase

### Notes about this data:
- Some columns have NAs! ML algorithms can NOT use features with NAs, so these will need to be imputed.
- We have binary, continuous, and categorical features in this dataset. A perk of ML models is that they can integrate multiple datatypes in a single model. 
- However, before being used as input, a categorical feature needs to be converted into set binary features using an approach called [one-hot-encoding](https://www.kaggle.com/dansbecker/using-categorical-data-with-one-hot-encoding). 

### The ML_preprocessing.py script automates all this data cleaning. 

ML_preprocessing.py will:
1. Drop columns if the number of NAs is above a certain threshold (default = 50%)
2. Impute the remaining columns using the median, mean, or mode of the data present for that feature
3. One-hot-encode  categorical feature

In [43]:
# B. Drop/Impute NAs and one-hot-encode categorical features

%run ../ML_preprocess.py -df data.txt -na_method median

Snapshot of input data...
                Class  Crubella_183_v1.0.csv  FamilySize FamilySize_cat  \
YP_008563134      gen                    0.0    0.010582         medium   
XP_010327628      gen                    0.0    0.000000          small   
XP_010327620  special                    0.0    0.052910         medium   
XP_010327578      gen                    0.0         NaN            NaN   
XP_010327494      gen                    1.0    0.021164         medium   

              Transferase  SQS_PSY  Exo_endo_phos  IPT  Paxillaris_medKaKs  \
YP_008563134          0.0      NaN            0.0  0.0            0.179007   
XP_010327628          NaN      NaN            0.0  0.0            0.179007   
XP_010327620          0.0      NaN            0.0  0.0            0.237404   
XP_010327578          0.0      NaN            0.0  0.0            0.237404   
XP_010327494          0.0      0.0            0.0  0.0            0.092387   

              GalKase_gal_bdg      ...        Polysacc

### Defining the testing set data

We want to set aside a subset of our data to use to test how well our model performed. Note that this is done before feature engineering, parameter selection, or model training. This will ensure our performance metric is entirely independent from our modeling!


### The test_set.py script

test_set.py will:
- randomly select a subset of the instances (i.e. rows) to use for testing
- For classification problems (i.e. this example), it will select the desired number (-n) or percent (-p) of instances from each class (specify -use if some classes shouldn't be in the test set, like unknown samples). 
- For regression problems (i.e. predicting plant height), it will randomly select the desired number (-n) or percent (-p) from the whole dataset. 

In [44]:
# C. Define test set

%run ../test_set.py -df data_mod.txt -use gen,special -type c -p 0.1 -save test_genes.txt

Holding out 10.0 percent
Pulling test set from classes: ['gen', 'special']
285 instances in test set
finished!


## 3. Feature engineering

While one major advantage of ML approaches is that they are robust when the number of features is very large, there are cases where removing unuseful features or selecting only the best features may help you better answer your question. One common issue we see with using feature selection for machine learning is using the whole dataset to select the best features, which results in overfitting! Be sure to define a test set before feature selection is performed. 


Feature_Selection.py allows you to perform feature selection using a number of different feature selection algorithms including:

- Enrichement (Fisher's Exact Test: fet) (for binary classification and binary features only)
- Random Forest (rf)
- L1/LASSO
- Relief (Need to install ReBATE to run)
- Bayesian LASSO (bl)(for regression only)
- Elastic Net (EN)(for regression only)


Here we will use one of the most common feature selection algorithms: LASSO. We will 

 LASSO selects all features that aren't assigned zero weight from the linear model, with the user able to specify the amount of sparcity (-p). For more information about what parameters are required for each algorithm, run Feature_Selection.py with no arguments:

%run ../Feature_Selection.py
First, let's use the LASSO algorithm to select the best feature for classifying the top 25% (1) from bottom 75% (0) of lines. Running LASSO on a classification problem, the sparcity parameter is required, where the smaller the number the fewer features will be selected.

We will try using -p 0.1 and -p 0.02...

%run ../Feature_Selection.py -alg lasso -df data_binary.txt -n 10 -y_name yield -type c -p 0.1 -save feat_lasso_0.1
%run ../Feature_Selection.py -alg lasso -df data_binary.txt -n 10 -y_name yield -type c -p 0.02 -save feat_lasso_0.02
Using sparcity = 0.01 (-p 0.01), we selected 63 features and lowering the sparcity (-p 0.02) drove that down to the top 15 features. You should now see a new file for each of these runs that contains a list of the features selected. We can use that directly as input into the ML pipeline!

We'll run the exact same binary classification pipeline as above, except this time we willl specify to only use the features from our feature selection (-feat)

%run ../ML_classification -df data_binary.txt -test test_bin -y_name yield \
-alg SVM -gs true -gs_reps 2 -n 2 -feat feat_lasso_0.02 -save data_binary_SVM_top15
When we ran SVM to classify lines using all 200 transcript features we got an F-measure (F1) = 0.747, so you can see that we did nearly as well just using the top 15 features.

In [23]:
%run ../Feature_Selection.py -df data_mod.txt -alg lasso -type c -n 10 -save top_feat_lasso.txt

Holding out 10.0 percent
Pulling test set from classes: ['gen', 'special']
285 instances in test set
finished!


By running the code above, you've generated three new files called test_reg, test_bin, and test_mc that should be in your working directory. These files contain the list of lines that will be held out of training and used to 

# Building Machine Learning Models


## 3. Regression models

The machine learning algorithms in the ML_Pipeline are implement from [SciKit-Learn](https://scikit-learn.org/stable/), which has excellent resources to learn more about the ins and outs of these algorithms.

Regression algorithms available in the pipeline are: Support Vector Regression with linear (SVM), polynomial (SVMpoly), and radial basis function (SVMrbf) kernels, Random Forest (RF), Gradient Tree Boosting (GB), Logistic Regression (LogReg).

Note, there are many functions available within the pipeline that are not described in this tutorial. Run python ML_regression.py without any arguments to see more details!

We'll start by using RF to predict maize grain yield using the test_reg 

In [ ]:
%run ../ML_regression.py -df data_regression.txt -test test_reg -y_name yield \
-alg RF -gs true -gs_reps 2 -n 2

**Results Breakdown**

First, note that you see two sets of results, first results from the validation set followed by results from the test. If this is your final model, you can report the results from the test set, however, if you are going to compare this model to, say a model using a different algorithm, and then select which algorithm to use for your study, you want to use the validation set results. Basically, you want to avoid making any decisions about the modeling process using the test set results because it may lead to overfitting.

For regression models, four performance metrics are reported ([MSE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error), [EVS](https://scikit-learn.org/stable/modules/model_evaluation.html#explained-variance-score), [r2](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html#sklearn.metrics.r2_score), and [PCC](https://docs.scipy.org/doc/numpy/reference/generated/numpy.corrcoef.html) along with the standard deviation (std) and standard error (se) over the replicates (n). In the case of regression models, each replicate is different because different [cross validation folds](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) are assigned to each replicate.

One parameter that can be adjusted is how many cross validation folds you want to include (-cv). The default and a commonly used fold number is -cv 10, however, if you have a small dataset, using fewer folds (-cv 5) may perform better. In the extreme case, if you have very few instances to train on, you can set -cv equal to the number of instances in your dataset, allowing you to perform leave one out [LOO](https://www.cs.cmu.edu/~schneide/tut5/node42.html) cross validation.

**Output files**

You will also note that the pipeline generated a number of output files from your run. You can specify a prefix for these files using -save. Here is a breakdown of what you'll find in those files:

- **"_results":** will give you a summary of the results from the run, similar to what you see when you run the pipeline on the command line.
- **"_scores":** For each instance you see the original value to predict (Y), the mean and stdev prediction across all replicates, and the predicted Y for each replicate
- **"_imp":** the importance of each feature in your model. For RF and GTB this score represents the [Gini Index](https://medium.com/the-artificial-impostor/feature-importance-measures-for-tree-models-part-i-47f187c1a2c3), while for LogReg and SVM it is the [coefficient](https://medium.com/@aneesha/visualising-top-features-in-linear-svm-with-scikit-learn-and-matplotlib-3454ab18a14d). SVM with non-linear kernels (i.e. poly, rbf) does not report importance scores.
- **"_GridSearch":** the average performance metric across the whole possible parameter space tested via the grid search. 

## 4. Binary classification models

All of the algorithms available for regression are also available for classification. The pipeline uses 1 and 0 as the default positive and negative classes, respectively. However, you can specify your own pos/neg classes using -pos POS_string -neg NEG_string. 


Lets try using SVM with a linear kernel to predict if a line is in the top 25% (pos) or bottom 75% (neg) for yield.

In [24]:
%run ../ML_classification -df data.txt -test test_genes.txt -cl_train special,gen -y_name Class -apply unknown -alg SVM -gs true -gs_reps 3 -n 10

Removing test instances to apply model on later...
Snapshot of data being used:
             Class  Exo_endo_phos  FamilySize  Transferase  IPT
XP_004250654   gen            0.0    0.033333          0.0  0.0
XP_010315005   gen            0.0    0.016667          0.0  0.0
XP_004232800   gen            0.0    0.008333          0.0  0.0
XP_004231928   gen            0.0    0.016667          0.0  0.0
XP_004236813   gen            0.0    0.008333          0.0  0.0


CLASSES: ['gen' 'special']
POS: special <class 'str'>
NEG: gen <class 'str'>

Balanced dataset will include 532 instances of each class


===>  Grid search started  <===
Round 1 of 3
Round 2 of 3
Round 3 of 3
Parameter sweep time: 10.905242 seconds
Parameters selected: Kernel=Linear, C=1.0
Grid search complete. Time: 10.908467 seconds


===>  ML Pipeline started  <===
  Round 1 of 10


ValueError: Found array with 0 sample(s) (shape=(0, 563)) while a minimum of 1 is required.

**Results Breakdown**

First, note that before the grid search started running, the model told you what your positive and negative class strings were and how many instances would be in each class for each replicate. This is an important feature in the ML-Pipeline. Training a ML classifier with unbalanced data, or data with different numbers of each instance, can cause your model to be biased toward predicting the more numerous class. For example, if you train a model using 100 positive examples and 100,000 negative examples, your model would do well to just call instance negative! Therefore, in this pipeline, the larger classes are randomly downsampled to generate balanced datasets. To ensure we still utilize as much data as possible, this downsampling is done independelty for each replicate (-n). 

A number of performance metrics are included for binary classification models, including [AUC-ROC](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html), [F-measure](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html), [AUC-PRC](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html), [accuracy](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html), and [precision](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html#sklearn.metrics.precision_score).

**Output files**

The classficiation pipeline generates similar output as the regression pipeline, although there are a few notable differences. 

- **"_scores":** Here, the predicted probability (pp) scores are reported for each replicate. The pp score represents how confident the model was in its classification, where a pp=1 means it is certain the instance is positive and pp=0 means it is certain the instance is negative. For each replicate, an instance is classified as pos if pp > threshold, which is defined as value between 0.001-0.999 that maximises the F-measure. While the performance metrics generated by the pipeline are calcuated for each replicate independently, we want to be able to make a final statement about which instances were called as positive and which were called as negative. You'll find those results in this file. To make this final call we calculated the mean threshold and the mean pp for each instance and called the instance pos if the mean pp > mean threshold. 
- **"_results":** Here you will see an overview of the results similar to what is printed in the command line. However, for classification problems you will see two additional sections: the Mean Balanced Confusion Matrix (CM and the Final Full CM. The mean balanced CM was generated by taking the average number of true positives (TP), true negatives (TN), false positives (FP), and false negatives (FN) across all replicate (which have been downsampled randomly to be balanced). The Final Full CM represents the final TP, TN, FP, and FN results from the final pos/neg classifications (descirbed above in _scores) for all instances in your input dataset. 
- **"_BalancedID":** Each row lists the instances that were included in each replicate (-n) after downsampling. 

## 5. Multi-class classification models

All of the algorithms available for regression and binary classification are also available for multiclass classification. There are no default classes for multiclass models, so you will have to specity what classes are including using (-cl_train)


Lets try using SVM with a linear kernel to predict if a line is in the top 25% (2) or middle 50% (1) or bottom 25% (0) for yield.

In [ ]:
%run ../ML_classification.py -df data_multiclass.txt -test test_mc -y_name yield \
-cl_train top,middle,bottom -alg SVM -gs true -gs_reps 2 -n 2 

**Results Breakdown**

The output for multi-class classification models is similar to binary classification models with a few key differences. 

*An important note: For binary classification using balanced datasets, you would expect a ML model that was just randomly guessing the class to be correct ~50% of the time, because of this the random expectation for performance metrics like AUC-ROC and the F-measure are 0.50. This is not the case for multi-class predictions. Using our model above as an example, a ML model that was randomly guessing top, middle, or bottom, would only be correct ~33% of the time. That means models performing with >33% accuracy are performing better than random expectation.*

There are two types of performance metrics for multi-class models, commonly referred to as macro and micro metrics. Micro performance metrics are generated for each class in your ML problem. For example, from our model we will get three micro F-measures (F1-top, F1-middle, F1-bottom). These micro scores are available in the *_results* output file. Macro performance metrics are generated by taking the average of all the micro performance metrics. These scores are what are printed in the command line. 

# Additional ML-Pipeline Features

## 6.A Data Preprocessing

Coming soon - imputing NAs, one-hot encoding, etc.

## 6.B Feature Selection

While one major advantage of ML approaches is that they are robust when the number of features is very large, there are cases where removing unuseful features or selecting only the best features may help you better answer your question. One common issue we see with using feature selection for machine learning is using the whole dataset to select the best features, which results in overfitting [see here](https://www.nature.com/articles/srep10312). The ML-Pipeline allows you to perform feature selection within the training-validation-testing scheme, thereby avoiding overfitting. 

There are many strategies for feature selection out there. The ML-Pipeline allows you to perform feature selection using:
- Enrichement (Fisher's Exact Test: fet) (for binary classification and binary features only)
- Random Forest (rf)
- L1/LASSO 
- Relief (Need to install [ReBATE](https://github.com/EpistasisLab/ReBATE) to run)
- Bayesian LASSO (bl)(for regression only)
- Bayes A (ba)(for regression only)
- Elastic Net (EN)(for regression only)
- ridge regression (rrblup)(for regression only)
- Random

For some algorithms (e.g. RF, rrBLUP, relief, random), the user specifies the number of features to be selected (-n 10), while other algorithms (e.g. LASSO, FET) select as many features that meet a criteria. For example, FET selects all features enriched in your positive class using the p-value cutoff you designate (-p 0.05) and LASSO selects all features that aren't assigned zero weight from the linear model, with the user able to specify the amount of sparcity (-p). For more information about what parameters are required for each algorithm, run Feature_Selection.py with no arguments:

In [ ]:
%run ../Feature_Selection.py

First, let's use the LASSO algorithm to select the best feature for classifying the top 25% (1) from bottom 75% (0) of lines. Running LASSO on a classification problem, the sparcity parameter is required, where the smaller the number the fewer features will be selected. 

We will try using -p 0.1 and -p 0.02...

In [ ]:
%run ../Feature_Selection.py -alg lasso -df data_binary.txt -n 10 -y_name yield -type c -p 0.1 -save feat_lasso_0.1
%run ../Feature_Selection.py -alg lasso -df data_binary.txt -n 10 -y_name yield -type c -p 0.02 -save feat_lasso_0.02

Using sparcity = 0.01 (-p 0.01), we selected 63 features and lowering the sparcity (-p 0.02) drove that down to the top 15 features. You should now see a new file for each of these runs that contains a list of the features selected. We can use that directly as input into the ML pipeline!

We'll run the exact same binary classification pipeline as above, except this time we willl specify to only use the features from our feature selection (-feat)

In [ ]:
%run ../ML_classification -df data_binary.txt -test test_bin -y_name yield \
-alg SVM -gs true -gs_reps 2 -n 2 -feat feat_lasso_0.02 -save data_binary_SVM_top15

When we ran SVM to classify lines using all 200 transcript features we got an F-measure (F1) = 0.747, so you can see that we did nearly as well just using the top 15 features. 

## 6.C Applying ML model to unknowns

Because the ML-Pipeline generates many many models to make one prediction (e.g. 100 replicates with 10-fold cross valiation is like having 1,000 seperate models) the pipeline does not "save" the models. However, you can still use the pipeline to apply your trained model to unknown instances. Unknown instances must have the same features as your training data. In this tutorial for example, we would need to know transcript levels for all of the same genes, but we would not need to know the yield before hand.

Start by simply adding your unknown instances to the input dataframe. In the column that has the value or class (Y/Class) for the known instances, put a different identifier (e.g. "unknown").

For this example, we've generated a version of the regression dataset (data_regression_unk.txt) where the first 20 lines were designated as unknowns.


In [ ]:
d_reg = pd.read_csv('data_regression_unk.txt', sep='\t', index_col = 0)
print(d_reg.ix[:8,:4])

Now, we can run the ML-Pipeline again, but this time specifying that we want to apply the models to the unknowns (-apply unknown)

In [ ]:
%run ../ML_regression.py -df data_regression_unk.txt -test test_reg -y_name yield \
-alg RF -gs true -gs_reps 2 -n 2 -apply unknown

To see the predicted yield for those five lines, check out the "_scores" file that was just generated. 

## 6.D Visualizing Your Results

There are a number of vizualization tools available in the ML-Pipeline. We will describe just some of them here!

**Generate Plots while training and testing your models**

1. One optional parameter for both the ML_classification.py and ML_regression.py pipelines is -plots T/F. The default is False, but if you want to generate some plots directly during your ML run, set -plots T. For classification you will get an [AUC-ROC plot](https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5) and an [AUC-PRC plot](https://classeval.wordpress.com/introduction/introduction-to-the-precision-recall-plot/). 

2. In the ML_classification.py pipeline, another plotting option is to auto-generate a [confusion matrix heatmap](https://towardsdatascience.com/understanding-confusion-matrix-a9ad42dcfd62) (-cm), just set -cm T

**Generate plots post-model building**
1. **ML_plots.py** allows you to generate AUC-ROC and AUC-PRC plots from multiple runs, for example if you wanted to compare performance of different algorithms or using different sets of features. For example, we can compare the results from our classification model with all 200 features with the one generated using the top 15 features:

```
ML_plots.py [SAVE_NAME] [POS] [NEG] [M1_name] [PATH_M1_scores] [M2_name] [PATH_M2_scores]... [Mn_name] [PATH_Mn_scores]
```

In [ ]:
%run ../ML_plots.py binary_plots 1 0 all data_binary.txt_SVM_scores.txt top15 data_binary_SVM_top15_scores.txt

2. **compare_classifiers.py** takes in the results from different classification models and generates a list of which instances were correctly and incorrectly predicted by each model and generates a venn diagram showing the overlap.

```
ML_plots.py -scores PATH_M1_scores,PATH_M2_scores,...,PATH_Mn_scores -ids M1_name,M2_name,...,Mn_name -save [SAVE_NAME]
```

In [ ]:
%run ../compare_classifiers.py  -scores data_binary.txt_SVM_scores.txt,data_binary_SVM_top15_scores.txt -ids all,top15 -save binary_comp

These results indicate that our models using all and the top 15 features both predicted 88 instances correctly as positive, but only 82 of those instances were the same for both models, meaning they each uniquely correctly classified 6 instances as positive.

3. **plot_predprob.py**